# DATA

In [1]:
%load_ext autoreload
%autoreload 2
import genericIO
import SepVector
import Hypercube
import numpy as np

import __pyDaskVector as DaskVector
import __pyDaskOperator as DaskOperator
import pyOperator as Operator

from dask_util import DaskClient

import time

/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:

n1 = 300
n2 = 50

d1 = 20
d2 = 10

o1 = 0
o2 = 0

slow = SepVector.getSepVector(Hypercube.hypercube(ns=[n1,n2],ds=[d1,d2],os=[o1,o2]),storage='dataComplex')

slowNd = slow.getNdArray()
slowNd[:] = 1/2000*1/2000
slowNd[int(1.5*n2/3)::,:] = 1/3000*1/3000

den = slow.clone()
den.set(1)

complexVector
Axis 1: n=300	o=0.000000	d=20.000000
Axis 2: n=50	o=0.000000	d=10.000000

In [3]:
param = {
    "n_workers" : 4,
    "processes" : True,
    "dashboard_address" : ":8989"
}
client = DaskClient(local_params=param)

In [4]:
client.getClient()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8989/status,
Dashboard: http://127.0.0.1:8989/status,Workers: 4
Total threads: 24,Total memory: 187.50 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38489,Workers: 4
Dashboard: http://127.0.0.1:8989/status,Total threads: 24
Started: Just now,Total memory: 187.50 GiB
Comm: tcp://127.0.0.1:42532,Total threads: 6
Dashboard: http://127.0.0.1:36286/status,Memory: 46.88 GiB
Nanny: tcp://127.0.0.1:39258,


In [5]:
d = DaskVector.DaskVector(client, from_vector=slow, chunks=(1,10), asynchronous=True)
m = d.clone()


In [6]:
d.fut

[<Future: finished, type: SepVector.complexVector, key: window-caf2b5b3-555c-473e-a80b-9d71e41b8f10>,
 <Future: finished, type: SepVector.complexVector, key: window-3175f8ae-0b9f-4944-813d-687552332bcf>,
 <Future: finished, type: SepVector.complexVector, key: window-e1704915-54b7-4145-8e35-8b080450ea04>,
 <Future: finished, type: SepVector.complexVector, key: window-4a4bd561-1167-4d65-918a-53c5ea7a5b1b>,
 <Future: finished, type: SepVector.complexVector, key: window-641f74b0-788b-4b9c-99d2-4f2319515384>,
 <Future: finished, type: SepVector.complexVector, key: window-d2abdf54-37f0-44a9-9c41-c9557384a080>,
 <Future: finished, type: SepVector.complexVector, key: window-1d804b36-009f-4b33-8e67-efd6f0af1a36>,
 <Future: finished, type: SepVector.complexVector, key: window-e4ce22a1-d372-4eee-9a10-5ca6eac9b293>,
 <Future: finished, type: SepVector.complexVector, key: window-c92ad76c-5be9-4cad-b940-b592aabe2424>,
 <Future: finished, type: SepVector.complexVector, key: window-d40241ae-a636-4774-

In [7]:
import time
class Random(Operator.Operator):

    def __init__(self, model, data):
        self.setDomainRange(model, data)

    def forward(self, add, model, data):
        time.sleep(np.random.randint(1,5))
        data[:] = model[:]

    def adjoint(self, add, model, data):
        time.sleep(np.random.randint(1,5))
        model[:] = data[:]

In [8]:
dask_random = DaskOperator.DaskOperator(client, Random, m, d)

In [9]:
d2 = d.clone()

In [10]:
# dask_random.forward(False, m, d)
# dask_random.forward(False, m, d2)

In [ ]:
dask_random.forward(False, m, d)
dask_random.adjoint(False, m, d)

/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.9/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is z